<a href="https://colab.research.google.com/github/shahchhatru/AI_colab_notebooks/blob/main/ExploringGNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.6 MB/s eta 0:00:00


In [2]:
import torch_geometric as tg
from torch_geometric.datasets import Planetoid


## Load the datasets

In [3]:
dataset=Planetoid(root="/" , name="Cora")
dataset

Processing...
Done!


Cora()

# Dataset properties

In [4]:
print(dataset)
print("No of graphs :\t\t",len(dataset))
print("num_classes",dataset.num_classes)
print(dataset.num_node_features)
print(dataset.num_edge_features)

Cora()
No of graphs :		 1
num_classes 7
1433
0


# Dataset shapes

In [5]:
print(dataset.data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


/usr/local/lib/python3.10/dist-packages/torch_geometric/data/in_memory_dataset.py:301: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [6]:
print("edge_index:\t\t",dataset.data.edge_index.shape)
print(dataset.data.edge_index)
print("\n")
print("train_mask:\t\t",dataset.data.train_mask.shape)
print(dataset.data.train_mask)
print("\n")
print("x:\t\t",dataset.data.x.shape)
print(dataset.data.x)
print("\n")
print("y:\t\t",dataset.data.y.shape)
print(dataset.data.y)

edge_index:		 torch.Size([2, 10556])
tensor([[ 633, 1862, 2582,  ...,  598, 1473, 2706],
        [   0,    0,    0,  ..., 2707, 2707, 2707]])


train_mask:		 torch.Size([2708])
tensor([ True,  True,  True,  ..., False, False, False])


x:		 torch.Size([2708, 1433])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


y:		 torch.Size([2708])
tensor([3, 4, 4,  ..., 3, 3, 3])


In [7]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv


In [8]:
data=dataset[0]

In [9]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [10]:
class Net(torch.nn.Module):
  def __init__(self):
    super(Net,self).__init__()

    self.conv=SAGEConv(dataset.num_features,
                       dataset.num_classes,
                       aggr="max") #max,mean,add


  def forward(self):
    x=self.conv(data.x,data.edge_index)
    return F.log_softmax(x,dim=1)


In [11]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
device

device(type='cpu')

In [13]:
model,data=Net().to(device),data.to(device)
model,data

(Net(
   (conv): SAGEConv(1433, 7, aggr=max)
 ),
 Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708]))

In [15]:
optimizer=torch.optim.Adam(model.parameters(),lr=0.01,weight_decay=5e-4)

In [16]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    weight_decay: 0.0005
)

In [17]:
def train():
  model.train()
  optimizer.zero_grad()
  F.nll_loss(model()[data.train_mask],data.y[data.train_mask]).backward()
  optimizer.step()

def test():
  model.eval()
  logits,accs=model(),[]
  for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)

  return accs


In [18]:
best_val_acc = test_acc = 0
for epoch in range(1,100):
    train()
    _, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Val: {:.4f}, Test: {:.4f}'

    if epoch % 10 == 0:
        print(log.format(epoch, best_val_acc, test_acc))

Epoch: 010, Val: 0.7200, Test: 0.7370
Epoch: 020, Val: 0.7200, Test: 0.7370
Epoch: 030, Val: 0.7200, Test: 0.7370
Epoch: 040, Val: 0.7200, Test: 0.7370
Epoch: 050, Val: 0.7200, Test: 0.7370
Epoch: 060, Val: 0.7200, Test: 0.7370
Epoch: 070, Val: 0.7220, Test: 0.7130
Epoch: 080, Val: 0.7220, Test: 0.7130
Epoch: 090, Val: 0.7220, Test: 0.7130
